The notebook requires a preprocessed dataset. This can be done either by executing the notebook
**data_preprocessing.ipynb** or by running the **data_preprocessing.py** Python script, which can be easily executed like so:
```
python data_preprocessing.py
```

Also, make sure that the **bnci_utils.py** file is in the same directory as this notebook as it contains part of the
functionality. The resulting preprocessed data should not be renamed unless you also change their names in the
notebook.

In [ ]:
import os
import numpy as np
import tensorflow as tf
import keras
import nengo_dl
from tensorflow.python.keras import Input, Model
import nengo
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.layers import Conv2D, Dropout, AveragePooling2D, Flatten, Dense, BatchNormalization, Conv3D
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras import backend as K
import pandas as pd
from sklearn import metrics
import bnci_utils as utils

Two models for the simulation are available - change the value of the **model** variable to either 'default_cnn' or
'p300_exp_cnn'. E.g:
```
model = 'p300_exp_cnn'
```

The name of the output files can be changed as well - change the value of the **iteration_data_file_name** variable to
rename the output file for each iteration, and the value of the **iteration_stats_file_name** to rename the output file
for statistics from the entire simulation. Note that both names **need** to have the .xlsx extension. E.g:

```
iteration_data_file_name = 'simulation_output.xlsx'
```

In [ ]:
num_participants = 18

# List of files with samples from each participant
dataset_path = os.path.join('dataset_result')
files = [os.path.join(dataset_path, 'P{:02d}.npz'.format(i+1)) for i in range(num_participants)] # P01 - P18 files

data_output_folder = 'output' # output path for statistics from the simulation
iteration_data_file_name = 'cnn_p300_individuals_data.xlsx' # file name of excel file with data from each iteration
iteration_stats_file_name = 'cnn_p300_individuals_stats.xlsx' # file name for statistics from the simulation (i.e

# List of tested models - default CNN is the originally tested network, p300_exp_cnn is the CNN from the P300 experiment
models = {
    'default_cnn': utils.cnn_model,
    'p300_exp_cnn': utils.original_p300_model
}

# Model function to create the model for simulation
model_fn = models['p300_exp_cnn']

In [28]:
# set seed to produce a consistent result
seed = 2
np.random.seed(seed)
tf.random.set_seed(seed)

In [29]:
def run_individual(file, particip_num, model, params_save_path, test_size=0.25, epochs=30, scale_firing_rates=1000,
                   synapse=0.01, timesteps=50):
    print(f'Running ANN and SNN for file: {file}')

    dataset = np.load(file) # load numpy file containing the preprocessed data for specific participant
    features, labels = dataset['features'], dataset['labels'] # get features and labels from the numpy file

    # Transform numpy arrays to be usable with Nengo
    features, labels = utils.reshape_dataset(features, labels)

    # Split the data into 75% training and 25% testing
    x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=test_size, random_state=seed,
                                                        shuffle=True)
    print('X (train) shape:', x_train.shape, 'Y (train) shape:', y_train.shape)
    print('X (test) shape:', x_test.shape, 'Y (test) shape:', y_test.shape)

    ann_stats = utils.run_ann(model, (x_train, y_train), (x_test, y_test), params_save_path, particip_num, num_epochs=epochs,
                              optimizer=keras.optimizers.Adam(), loss=keras.losses.BinaryCrossentropy(), batch_size=1,
                              callbacks=[EarlyStopping(monitor='loss', patience=8, verbose=1, restore_best_weights=True)])
    snn_stats = utils.run_snn(model, x_test,  y_test, params_save_path, particip_num, batch_size=1,
                      timesteps=timesteps, synapse=synapse, scale_firing_rates=scale_firing_rates)

    return ann_stats, snn_stats

In [30]:
ann, snn = [], []
participant_no = 1

params_save_dir = 'cnn_individuals_nengo_params'
os.makedirs(params_save_dir, exist_ok=True)

for file in files:
    file_name = 'P{:02d}'.format(participant_no)
    model = model_fn(seed=seed)
    params_save_path = os.path.join(params_save_dir, file_name)

    ann_stats, snn_stats = run_individual(file, participant_no, model, params_save_path)

    ann.append(ann_stats) # append statistics to the list
    snn.append(snn_stats)

    participant_no += 1 # increase participant number

    # Delete model and clear session to prevent memory leaks
    K.clear_session()
    del model

Running ANN and SNN for file: dataset_result\P01.npz
X (train) shape: (90, 1, 5040) Y (train) shape: (90, 1, 2)
X (test) shape: (30, 1, 5040) Y (test) shape: (30, 1, 2)
Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
Epoch 1/30
90/90 [==============================] - 4s 7ms/step - loss: 0.6808 - probe_loss: 0.6808 - probe_accuracy: 0.6134  

c:\dev\anaconda\envs\spiking-nn\lib\site-packages\nengo_dl\converter.py:326: UserWarning: Layer type Dropout does not have a registered converter. Falling back to TensorNode.
  warnings.warn(
c:\dev\anaconda\envs\spiking-nn\lib\site-packages\nengo_dl\converter.py:326: UserWarning: average_pooling2d.padding has value same != valid, which is not supported. Falling back to TensorNode.
  warnings.warn(
c:\dev\anaconda\envs\spiking-nn\lib\site-packages\nengo_dl\converter.py:326: UserWarning: Cannot convert BatchNormalization layer to native Nengo objects unless inference_only=True or layer.trainable=False. Falling back to TensorNode.
  warnings.warn(
c:\dev\anaconda\envs\spiking-nn\lib\site-packages\nengo_dl\converter.py:588: UserWarning: Activation type <function softmax_v2 at 0x000001A757CC1A60> does not have a native Nengo equivalent; falling back to a TensorNode
  warnings.warn(
c:\dev\anaconda\envs\spiking-nn\lib\site-packages\nengo_dl\simulator.py:1765: UserWarning: Number of elements

In [31]:
df = utils.create_data_df(ann, snn, num_participants)

# Rename iterations to participants
df = df.rename(columns={'iterations': 'participant'})

df

,participant,ann_accuracy,ann_precision,ann_recall,ann_f1,snn_accuracy,snn_precision,snn_recall,snn_f1
0,1,0.400000,0.500000,0.166667,0.250000,0.400000,0.500000,0.222222,0.307692
1,2,0.404762,0.368421,0.350000,0.358974,0.333333,0.250000,0.200000,0.222222
2,3,0.476190,0.476190,0.476190,0.476190,0.380952,0.368421,0.333333,0.350000
3,4,0.476190,0.428571,0.300000,0.352941,0.500000,0.461538,0.300000,0.363636
4,5,0.571429,0.555556,0.500000,0.526316,0.523810,0.500000,0.450000,0.473684
5,6,0.547619,0.600000,0.521739,0.558140,0.547619,0.611111,0.478261,0.536585
6,7,0.523810,0.533333,0.380952,0.444444,0.523810,0.533333,0.380952,0.444444
7,8,0.380952,0.363636,0.400000,0.380952,0.428571,0.409091,0.450000,0.428571
8,9,0.428571,0.363636,0.190476,0.250000,0.404762,0.333333,0.190476,0.242424
9,10,0.357143,0.363636,0.380952,0.372093,0.404762,0.400000,0.380952,0.390244


In [32]:
os.makedirs(data_output_folder, exist_ok=True)
df.to_excel(os.path.join(data_output_folder, iteration_data_file_name))

'Statistics for iterations successfully saved.'

'Statistics for iterations successfully saved.'

In [33]:
df_stats = utils.create_stats_df(df)
df_stats

,models,average_accuracy,max_accuracy,accuracy_std,average_precision,max_precision,average_recall,max_recall,average_f1,max_f1
0,ann,0.460053,0.571429,0.059164,0.455619,0.60,0.390144,0.523810,0.413017,0.558140
1,snn,0.473280,0.619048,0.074170,0.464604,0.65,0.395785,0.590909,0.421364,0.619048


In [34]:
# Save the statistics
df_stats.to_excel(os.path.join(data_output_folder, iteration_stats_file_name))

'File with statistics successfully saved.'

'File with statistics successfully saved.'

In [35]:
# Print confusion matrices
utils.print_confusion_matrices(ann, snn)


Confusion matrices for the ANN:
[[ 9  3]
 [15  3]] 

[[10 12]
 [13  7]] 

[[10 11]
 [11 10]] 

[[14  8]
 [14  6]] 

[[14  8]
 [10 10]] 

[[11  8]
 [11 12]] 

[[14  7]
 [13  8]] 

[[ 8 14]
 [12  8]] 

[[14  7]
 [17  4]] 

[[ 7 14]
 [13  8]] 

[[12 10]
 [10 10]] 

[[13  7]
 [16  6]] 

[[10 13]
 [11  8]] 

[[11  9]
 [14  8]] 

[[11 10]
 [10 11]] 

[[ 9 10]
 [13 10]] 

[[ 8 13]
 [10 11]] 

[[12 11]
 [13  6]] 

Confusion matrices for the SNN
[[ 8  4]
 [14  4]] 

[[10 12]
 [16  4]] 

[[ 9 12]
 [14  7]] 

[[15  7]
 [14  6]] 

[[13  9]
 [11  9]] 

[[12  7]
 [12 11]] 

[[14  7]
 [13  8]] 

[[ 9 13]
 [11  9]] 

[[13  8]
 [17  4]] 

[[ 9 12]
 [13  8]] 

[[13  9]
 [10 10]] 

[[13  7]
 [17  5]] 

[[13 10]
 [ 9 10]] 

[[13  7]
 [ 9 13]] 

[[ 8 13]
 [ 9 12]] 

[[10  9]
 [13 10]] 

[[10 11]
 [ 9 12]] 

[[13 10]
 [13  6]] 

